### This notebook is built to combine what've scraped (race result, form record, sectional time and racecard if available) into one dataset

#### Most of the feature engineering are performed in __form record__. Yet, there are 3 things needed to be merged from race_result and sectional time to form record
#### 1) Number of partcipating horses (for normalization)
#### 2) Prize money
#### 3) Section time

##### PS, Some races or even race day could be skipped when crawling. Not sure if because my computer do not have enough ram or whatever other reasons (e.g. the website is blocking).
##### Some steps below can help ensure the completeness of the data.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_rows', 100)

In [2]:
form_record = pd.read_csv('form_record.csv')
race_result = pd.read_csv('race_result.csv', dtype = {'date': str, 'match': str})

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
form_record.head()

,RaceIndex,Pla,Date,RC/Track/Course,Dist,Ground,RaceClass,Draw,Rating,Trainer,Jockey,LBW,WinOdds,ActWt,RunPo,FinishTime,Declare_Horse_Wt,Gear,VideoReplay,horseid
0,356,12,2018-01-21,"ST / Turf / ""A""",2000.0,G,5,12,35,A Lee,M L Yeung,7-3/4,16,126.0,13 13 13 13 12,2.05.83,1095,B/TT,NaN,HK_2012_P405
1,210,03,2017-11-22,"HV / Turf / ""C+3""",2200.0,G,5,6,35,A Lee,M F Poon,1-1/4,8.7,121.0,12 11 10 8 3 3,2.18.54,1095,B/TT,NaN,HK_2012_P405
2,145,01,2017-10-29,"HV / Turf / ""A""",2200.0,GF,5,5,30,A Lee,M L Yeung,1-1/4,5.8,128.0,9 6 5 5 5 1,2.18.77,1086,B/TT,NaN,HK_2012_P405
3,92,04,2017-10-08,"ST / Turf / ""B+2""",2000.0,G,5,2,31,A Lee,M F Poon,2-1/2,7.9,120.0,4 7 5 6 4,2.03.98,1082,B/TT,NaN,HK_2012_P405
4,2,09,2017-09-03,"ST / Turf / ""B""",1600.0,G,5,10,32,A Lee,M F Poon,4,40,118.0,11 9 8 9,1.36.80,1084,B/TT,NaN,HK_2012_P405


In [4]:
form_record['formatted_date'] = pd.to_datetime(form_record['Date'], format='%Y-%m-%d')

form_record.head()

,RaceIndex,Pla,Date,RC/Track/Course,Dist,Ground,RaceClass,Draw,Rating,Trainer,...,LBW,WinOdds,ActWt,RunPo,FinishTime,Declare_Horse_Wt,Gear,VideoReplay,horseid,formatted_date
0,356,12,2018-01-21,"ST / Turf / ""A""",2000.0,G,5,12,35,A Lee,...,7-3/4,16,126.0,13 13 13 13 12,2.05.83,1095,B/TT,NaN,HK_2012_P405,2018-01-21
1,210,03,2017-11-22,"HV / Turf / ""C+3""",2200.0,G,5,6,35,A Lee,...,1-1/4,8.7,121.0,12 11 10 8 3 3,2.18.54,1095,B/TT,NaN,HK_2012_P405,2017-11-22
2,145,01,2017-10-29,"HV / Turf / ""A""",2200.0,GF,5,5,30,A Lee,...,1-1/4,5.8,128.0,9 6 5 5 5 1,2.18.77,1086,B/TT,NaN,HK_2012_P405,2017-10-29
3,92,04,2017-10-08,"ST / Turf / ""B+2""",2000.0,G,5,2,31,A Lee,...,2-1/2,7.9,120.0,4 7 5 6 4,2.03.98,1082,B/TT,NaN,HK_2012_P405,2017-10-08
4,2,09,2017-09-03,"ST / Turf / ""B""",1600.0,G,5,10,32,A Lee,...,4,40,118.0,11 9 8 9,1.36.80,1084,B/TT,NaN,HK_2012_P405,2017-09-03


In [5]:
race_result.head()

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
0,1,7.0,TELEPHATIA(P405),M L Yeung,A Lee,123,1049,13,,12 13 13 ...,1:49.08,54,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2012_P405
1,2,1.0,NAMJONG TURBO(N250),K C Leung,P O'Sullivan,129,1079,11,SH,6 6 7 ...,1:49.08,5.4,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2011_N250
2,3,12.0,HEAR THE ROAR(M152),M Guyon,S Woods,118,1192,1,2,9 10 9 ...,1:49.38,3,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2010_M152
3,4,5.0,CASA JUNIOR(M366),K Teetan,T K Ng,127,1062,9,2,5 5 5 ...,1:49.40,55,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2010_M366
4,5,8.0,JOYFUL MISSION(S094),O Doleuze,R Gibson,123,1091,2,2,10 9 10 ...,1:49.40,32,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2013_S094


In [6]:
# https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2016/10/26&Racecourse=HV&RaceNo=4

# should contain number only
display(race_result[race_result['date'].str.contains('[a-zA-Z]')])

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
18257,VOID,1.0,BUDDY BUNDY(T157),N Callan,D Cruz,,,3,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2014_T157,NaN,NaN
18258,VOID,2.0,FLYING MONKEY(T361),J Moreira,W Y So,,,11,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2014_T361,NaN,NaN
18259,VOID,3.0,JUMBO HAPPINESS(V315),C Y Ho,C Fownes,,,2,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V315,NaN,NaN
18260,VOID,4.0,D B PIN(V344),Z Purton,J Size,,,4,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V344,NaN,NaN
18261,VOID,5.0,THE FULL BLOOM(V243),C Schofield,C H Yip,,,6,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V243,NaN,NaN
18262,VOID,6.0,BREEDERS' STAR(V197),D Whyte,D E Ferraris,,,1,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V197,NaN,NaN
18263,VOID,7.0,VERY RICH MAN(V286),N Rawiller,T K Ng,,,12,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V286,NaN,NaN
18264,VOID,8.0,FOX SUNTER(V221),H W Lai,K W Lui,,,10,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V221,NaN,NaN
18265,VOID,9.0,ISAAC(V387),S Clipperton,J Moore,,,7,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V387,NaN,NaN
18266,VOID,10.0,LUCKY SEVEN(V201),O Bosson,P O'Sullivan,,,5,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V201,NaN,NaN


In [7]:
# Check if it is the only match that VOID
display(race_result[race_result['Plc'] == 'VOID'])

# Drop VOID races
race_result = race_result.loc[race_result['Plc'] != 'VOID']

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
18257,VOID,1.0,BUDDY BUNDY(T157),N Callan,D Cruz,,,3,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2014_T157,NaN,NaN
18258,VOID,2.0,FLYING MONKEY(T361),J Moreira,W Y So,,,11,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2014_T361,NaN,NaN
18259,VOID,3.0,JUMBO HAPPINESS(V315),C Y Ho,C Fownes,,,2,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V315,NaN,NaN
18260,VOID,4.0,D B PIN(V344),Z Purton,J Size,,,4,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V344,NaN,NaN
18261,VOID,5.0,THE FULL BLOOM(V243),C Schofield,C H Yip,,,6,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V243,NaN,NaN
18262,VOID,6.0,BREEDERS' STAR(V197),D Whyte,D E Ferraris,,,1,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V197,NaN,NaN
18263,VOID,7.0,VERY RICH MAN(V286),N Rawiller,T K Ng,,,12,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V286,NaN,NaN
18264,VOID,8.0,FOX SUNTER(V221),H W Lai,K W Lui,,,10,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V221,NaN,NaN
18265,VOID,9.0,ISAAC(V387),S Clipperton,J Moore,,,7,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V387,NaN,NaN
18266,VOID,10.0,LUCKY SEVEN(V201),O Bosson,P O'Sullivan,,,5,,,2016/10/26,4,Class 4 - 1000M - (60-40),"HK$ 840,000",HK_2015_V201,NaN,NaN


In [8]:
# Still ran into error in transforming the date into datetime object: pd.to_datetime(race_result['date'], format = '%Y/%m/%d')

# https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2019/03/23
# Only races not in HK
# No odds data for the races
display(race_result.loc[race_result['date'].str.len() != 10])

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
43015,1,4.0,NORDIC WARRIOR(B371),M Chadwick,R Gibson,122,1068,2,,4 3 1,1:09.28,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2017_B371,NaN
43016,2,2.0,SMART CHARADE(A314),A Sanna,W Y So,125,1155,4,1 1/4,6 5 2,1:09.46,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2016_A314,NaN
43017,3,3.0,GALLANT RETURN(A308),Z Purton,D J Hall,122,1074,1,3 1/4,2 1 3,1:09.76,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2016_A308,NaN
43018,4,1.0,HAPPY TOUR(B374),C Schofield,C S Shum,125,1214,6,5 1/2,1 2 4,1:10.15,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2017_B374,NaN
43019,5,6.0,EMPIRE OF PATCH(A319),S de Sousa,J Moore,120,1122,3,9 1/4,3 4 5,1:10.77,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2016_A319,NaN
43020,6,5.0,MY DEAR(B445),K Teetan,A S Cruz,120,1022,5,17,5 6 6,1:11.98,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2017_B445,NaN
43021,1,6.0,DRAGON WARRIOR(V202),C Schofield,K H Ting,127,1198,8,,9 9 5 ...,1:49.60,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2015_V202,NaN
43022,2,9.0,WEALTHY FORTUNE(P324),S de Sousa,K L Man,116,1009,4,1 3/4,5 4 4 ...,1:49.87,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2012_P324,NaN
43023,3,3.0,PEARL FURU(T431),N Callan,P O'Sullivan,132,1162,9,3,6 6 6 ...,1:50.07,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2014_T431,NaN
43024,4,10.0,SURE SUPREME(B233),H N Wong,W Y So,112,1128,10,4 1/2,2 3 3 ...,1:50.31,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2017_B233,NaN


In [9]:
# shfit 2019/03/23 races by one columns start from Win Odds
# get 2019/03/23 races
shift = race_result.loc[(race_result['date'].str.contains('[a-zA-Z]')) | (race_result['date'].str.len() != 10)]

# drop the 2019/03/23 races first in race_result
race_result = race_result.loc[~((race_result['date'].str.contains('[a-zA-Z]')) | (race_result['date'].str.len() != 10))]

# Check if the drop works (should be blank)
print('Check if the drop works (should be blank)')
display(race_result.loc[(race_result['date'].str.contains('[a-zA-Z]')) | (race_result['date'].str.len() != 10)])
print('\n')

# shift
shift[['Win_Odds', 'date', 'match', 'cdr', 'prize_money', 'horseid']] = shift[['Win_Odds', 'date', 'match', 'cdr', 'prize_money', 'horseid']].shift(1, axis=1)

display(shift)

# concat back to race_result
race_result = pd.concat([race_result, shift], ignore_index=True)

Check if the drop works (should be blank)


,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid


,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
43015,1,4.0,NORDIC WARRIOR(B371),M Chadwick,R Gibson,122,1068,2,,4 3 1,1:09.28,NaN,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2017_B371
43016,2,2.0,SMART CHARADE(A314),A Sanna,W Y So,125,1155,4,1 1/4,6 5 2,1:09.46,NaN,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2016_A314
43017,3,3.0,GALLANT RETURN(A308),Z Purton,D J Hall,122,1074,1,3 1/4,2 1 3,1:09.76,NaN,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2016_A308
43018,4,1.0,HAPPY TOUR(B374),C Schofield,C S Shum,125,1214,6,5 1/2,1 2 4,1:10.15,NaN,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2017_B374
43019,5,6.0,EMPIRE OF PATCH(A319),S de Sousa,J Moore,120,1122,3,9 1/4,3 4 5,1:10.77,NaN,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2016_A319
43020,6,5.0,MY DEAR(B445),K Teetan,A S Cruz,120,1022,5,17,5 6 6,1:11.98,NaN,2019/03/23,1,Restricted Race - 1200M,"HK$ 2,000,000",HK_2017_B445
43021,1,6.0,DRAGON WARRIOR(V202),C Schofield,K H Ting,127,1198,8,,9 9 5 ...,1:49.60,NaN,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2015_V202
43022,2,9.0,WEALTHY FORTUNE(P324),S de Sousa,K L Man,116,1009,4,1 3/4,5 4 4 ...,1:49.87,NaN,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2012_P324
43023,3,3.0,PEARL FURU(T431),N Callan,P O'Sullivan,132,1162,9,3,6 6 6 ...,1:50.07,NaN,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2014_T431
43024,4,10.0,SURE SUPREME(B233),H N Wong,W Y So,112,1128,10,4 1/2,2 3 3 ...,1:50.31,NaN,2019/03/23,2,Class 5 - 1800M - (40-0),"HK$ 690,000",HK_2017_B233


In [10]:
# Check if the concat and shift works (should be blank)
display(race_result.loc[(race_result['date'].str.contains('[a-zA-Z]')) | (race_result['date'].str.len() != 10)])

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid


In [11]:
# create date for merge key later
race_result.loc[:, 'formatted_date'] = pd.to_datetime(race_result['date'], format = '%Y/%m/%d')
race_result.head()

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid,formatted_date
0,1,7.0,TELEPHATIA(P405),M L Yeung,A Lee,123,1049,13,,12 13 13 ...,1:49.08,54,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2012_P405,2015-01-01
1,2,1.0,NAMJONG TURBO(N250),K C Leung,P O'Sullivan,129,1079,11,SH,6 6 7 ...,1:49.08,5.4,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2011_N250,2015-01-01
2,3,12.0,HEAR THE ROAR(M152),M Guyon,S Woods,118,1192,1,2,9 10 9 ...,1:49.38,3,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2010_M152,2015-01-01
3,4,5.0,CASA JUNIOR(M366),K Teetan,T K Ng,127,1062,9,2,5 5 5 ...,1:49.40,55,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2010_M366,2015-01-01
4,5,8.0,JOYFUL MISSION(S094),O Doleuze,R Gibson,123,1091,2,2,10 9 10 ...,1:49.40,32,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2013_S094,2015-01-01


In [12]:
# Merge / Check Key

form_record['key'] = form_record['horseid'] + '_' + form_record['formatted_date'].astype(str)
race_result['key'] = race_result['horseid'] + '_' + race_result['formatted_date'].astype(str)

# check any duplicate (Horse should not run two races in a day)
# print(form_record['key'].value_counts())
# print(race_result['key'].value_counts())
print(form_record['key'].value_counts().reset_index(name="count").query("count > 1"))
print(race_result['key'].value_counts().reset_index(name="count").query("count > 1"))

Empty DataFrame
Columns: [index, count]
Index: []
Empty DataFrame
Columns: [index, count]
Index: []


In [13]:
# Check if any missing in data

# check any horse is missing first
# It should be blank
print('form_record is missing:', list(set(race_result['horseid'].unique()) - set(form_record['horseid'].unique())), '\n')
print('race_result is missing:', list(set(form_record['horseid'].unique()) - set(race_result['horseid'].unique())))

form_record is missing: [] 

race_result is missing: [nan]


In [14]:
# Could be upcoming races
# Entries of this runner: ...
form_record.loc[form_record['horseid'].isna()]

,RaceIndex,Pla,Date,RC/Track/Course,Dist,Ground,RaceClass,Draw,Rating,Trainer,...,WinOdds,ActWt,RunPo,FinishTime,Declare_Horse_Wt,Gear,VideoReplay,horseid,formatted_date,key
82338,22/09/2021 H HANDICAP Section 1,HK_2017_B288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [15]:
# drop upcoming races
form_record = form_record.loc[~form_record['horseid'].isna()]

print('form_record is missing:', list(set(race_result['horseid'].unique()) - set(form_record['horseid'].unique())), '\n')
print('race_result is missing:', list(set(form_record['horseid'].unique()) - set(race_result['horseid'].unique())))

form_record is missing: [] 

race_result is missing: []


In [16]:
# check if any races between 20150101 to present is missing
form_record_after2015 = form_record.loc[form_record['formatted_date'] >= np.datetime64('2015-01-01')]

# what race_result has but form_record_tmp hasn't
form_record_miss = list(set(race_result['key'].unique()) - set(form_record_after2015['key'].unique()))

# what form_record_tmp has but race_result hasn't
race_result_miss = list(set(form_record_after2015['key'].unique()) - set(race_result['key'].unique()))

# form record use horseid as crawling key (blank is expected)
print('form_record has to crawl on:\n', list(race_result.loc[race_result['key'].isin(form_record_miss)]['horseid'].unique()))

# race result use date as crawling key (blank is expected)
print('\nrace_result has to crawl on:\n', list(form_record_after2015.loc[form_record_after2015['key'].isin(race_result_miss)]['formatted_date'].dt.strftime('%Y/%m/%d').unique()))

form_record has to crawl on:
 ['HK_2019_D318', 'HK_2020_E227', 'HK_2018_C027', 'HK_2020_E129']

race_result has to crawl on:
 []


In [17]:
# WV matches that didn't appear in form_record

# If the date is upcoming races, drop and proceed
print(form_record_miss)

race_result = race_result.loc[~race_result['key'].isin(form_record_miss)]

['HK_2020_E129_2021-09-19', 'HK_2020_E227_2021-09-19', 'HK_2018_C027_2021-09-19', 'HK_2019_D318_2021-09-19']


In [18]:
# Find all the date of historical races before 2015/01/01 in which horses in our intereseted period are involved
# You can think of we are building a database that contain a full record of our interested horses.

race_result_comp = form_record.loc[form_record['formatted_date'] < np.datetime64('2015-01-01')]

# races that needed to be crawl as complement
print("race_result_comp:", list(race_result_comp['formatted_date'].dt.strftime('%Y/%m/%d').sort_values().unique()))

# number of races
len(race_result_comp['formatted_date'].unique())

race_result_comp: ['2007/12/15', '2008/02/09', '2008/03/09', '2008/09/21', '2008/10/19', '2008/10/26', '2008/11/08', '2008/11/16', '2008/11/30', '2009/01/01', '2009/01/04', '2009/01/07', '2009/01/28', '2009/02/11', '2009/02/15', '2009/02/22', '2009/03/01', '2009/03/07', '2009/03/15', '2009/03/22', '2009/03/28', '2009/04/05', '2009/04/13', '2009/04/22', '2009/04/26', '2009/05/01', '2009/05/06', '2009/05/17', '2009/05/21', '2009/05/31', '2009/06/03', '2009/06/07', '2009/06/13', '2009/06/21', '2009/06/28', '2009/07/01', '2009/09/13', '2009/09/20', '2009/10/01', '2009/10/07', '2009/10/11', '2009/10/17', '2009/10/21', '2009/10/25', '2009/11/01', '2009/11/04', '2009/11/07', '2009/11/15', '2009/11/18', '2009/11/22', '2009/11/25', '2009/11/29', '2009/12/02', '2009/12/06', '2009/12/09', '2009/12/13', '2009/12/16', '2009/12/19', '2009/12/23', '2009/12/27', '2010/01/01', '2010/01/06', '2010/01/10', '2010/01/13', '2010/01/16', '2010/01/20', '2010/01/31', '2010/02/03', '2010/02/07', '2010/02/10', '

467

### After crawling complement race result

In [19]:
# check on comp
race_result_comp = pd.read_csv('/Users/lauchunlok/HKJC-Scraping/clean/race_result_comp.csv', dtype = {'date': str, 'match': str})

# check anomaly: date has character or length is no equal to 10
race_result_comp.loc[(race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10)]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
0,1,11.0,ARISTOCRACY(CG070),Z Purton,C S Shum,120,1104,11,,1:41.20,16,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2005_G070,NaN
1,2,4.0,COULDN'T CARE LESS(CD009),S Dye,A Schutz,126,1080,1,NOSE,1:41.20,4.2,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2003_D009,NaN
2,3,5.0,AS YOU WISH(CE290),A Delpech,B K Ng,125,1048,7,N,1:41.30,12,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2004_E290,NaN
3,4,7.0,NOBLE ZOOM(CC298),P H Lo,A Lee,118,1084,5,1 1/2,1:41.40,11,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2002_C298,NaN
4,5,12.0,MASTER YING(CB249),E Saint Martin,T K Ng,119,1166,4,1 1/2,1:41.40,9.3,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2001_B249,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12260,NaN,10.0,BILLION(CH316),Y T Cheng,K W Lui,,,13,,,2010/05/30,11,Class 2 - 1600M - (100-80),"HK$ 1,150,000",HK_2006_H316,NaN,NaN
12261,NaN,11.0,JOLLY GOOD(L134),W M Lai,S Woods,,,10,,,2010/05/30,11,Class 2 - 1600M - (100-80),"HK$ 1,150,000",HK_2009_L134,NaN,NaN
12262,NaN,12.0,JOHNSON(J305),M Du Plessis,D J Hall,,,14,,,2010/05/30,11,Class 2 - 1600M - (100-80),"HK$ 1,150,000",HK_2007_J305,NaN,NaN
12263,NaN,13.0,PACKING BUDDY(K264),T H So,L Ho,,,6,,,2010/05/30,11,Class 2 - 1600M - (100-80),"HK$ 1,150,000",HK_2008_K264,NaN,NaN


In [20]:
# some date go into the column Win_Odds and some go into the column FinishTime
# and look into why

# races that are too old that running position is not recorded
print(race_result_comp.loc[(race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10)]['Win_Odds'].value_counts())

# 2010/05/30
# This race has been abandoned or not offered
# https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2010/05/30&Racecourse=ST&RaceNo=10
print(race_result_comp.loc[(race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10)]['FinishTime'].value_counts())

2008/02/09    150
2008/03/09    140
2007/12/15    138
10             14
11             14
Name: Win_Odds, dtype: int64
99            32
2010/05/30    28
13            13
11            12
17            12
              ..
79             1
64             1
85             1
1.9            1
4.3            1
Name: FinishTime, Length: 147, dtype: int64


In [21]:
# dropping 2010/05/30
race_result_comp.dropna(subset=['Plc'], inplace=True) # not shiftable

# shift old races that do not have running position by one columns start from running position
# get old races
old_race_result_comp = race_result_comp.loc[(race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10)]

# drop the old races first in race_result_comp
race_result_comp = race_result_comp.loc[~((race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10))]

print('Check if the drop works (should be blank):\n')
display(race_result_comp.loc[(race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10)])

# shift
old_race_result_comp[['RunningPosition', 'FinishTime', 'Win_Odds', 'date', 
                      'match', 'cdr', 'prize_money', 'horseid']] = old_race_result_comp[['RunningPosition', 'FinishTime', 'Win_Odds', 'date', 
                                                                                         'match', 'cdr', 'prize_money', 'horseid']].shift(1, axis=1)
print('Check if the shift works:\n')
display(old_race_result_comp)


# concat back to race_result_comp
race_result_comp = pd.concat([race_result_comp, old_race_result_comp], ignore_index=True)
# display to check again
print('Check if the concat and shift works (should be blank):\n')
display(race_result_comp.loc[(race_result_comp['date'].str.contains('[a-zA-Z]')) | (race_result_comp['date'].str.len() != 10)])

Check if the drop works (should be blank):



,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid


Check if the shift works:



,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid
0,1,11.0,ARISTOCRACY(CG070),Z Purton,C S Shum,120,1104,11,,NaN,1:41.20,16,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2005_G070
1,2,4.0,COULDN'T CARE LESS(CD009),S Dye,A Schutz,126,1080,1,NOSE,NaN,1:41.20,4.2,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2003_D009
2,3,5.0,AS YOU WISH(CE290),A Delpech,B K Ng,125,1048,7,N,NaN,1:41.30,12,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2004_E290
3,4,7.0,NOBLE ZOOM(CC298),P H Lo,A Lee,118,1084,5,1 1/2,NaN,1:41.40,11,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2002_C298
4,5,12.0,MASTER YING(CB249),E Saint Martin,T K Ng,119,1166,4,1 1/2,NaN,1:41.40,9.3,2007/12/15,1,Class 5 - 1650M - (40-10),"HK$ 465,000",HK_2001_B249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,10,9.0,BODACIOUS(CD170),M W Leung,Y S Tsui,116,1060,6,6 1/4,NaN,1:23.40,28,2008/03/09,11,Class 3 - 1400M - (80-60),"HK$ 780,000",HK_2003_D170
424,11,7.0,CIRCUIT HERO(CG304),O Doleuze,A Lee,120,1058,3,6 1/2,NaN,1:23.40,9.2,2008/03/09,11,Class 3 - 1400M - (80-60),"HK$ 780,000",HK_2005_G304
425,12,11.0,SCOOT DA LOOT(CH274),F Coetzee,A Schutz,118,962,13,6 3/4,NaN,1:23.40,24,2008/03/09,11,Class 3 - 1400M - (80-60),"HK$ 780,000",HK_2006_H274
426,13,5.0,MAGIC TURBO(J098),E Wilson,T W Leung,122,1034,1,7 3/4,NaN,1:23.60,98,2008/03/09,11,Class 3 - 1400M - (80-60),"HK$ 780,000",HK_2007_J098


Check if the concat and shift works (should be blank):



,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid


In [22]:
# race_result_comp merge key
race_result_comp.loc[:, 'formatted_date'] = pd.to_datetime(race_result_comp['date'], format = '%Y/%m/%d')
race_result_comp['key'] = race_result_comp['horseid'] + '_' + race_result_comp['formatted_date'].astype(str)

# check missing
# race result comp should have no missing
form_record_before2015 = form_record[form_record['formatted_date'] < np.datetime64('2015-01-01')]

race_result_comp_miss = list(set(form_record_before2015['key'].unique()) - set(race_result_comp['key'].unique()))

print('race_result_comp is missing:', race_result_comp_miss)

# If not blank, use below to get date that race_result has to crawl on 
# form_record_tmp[form_record_tmp['key'].isin(race_result_comp_miss)]['formatted_date'].dt.strftime('%Y/%m/%d').unique()

race_result_comp is missing: []


In [23]:
race_result_full = pd.concat([race_result, race_result_comp], ignore_index=True)

# race_result_full.to_csv('race_result_full.csv', index=False)

# create prize_money table
# match might be read as int or str sometime. So merge is mostly relied on key
prize_money_table = race_result_full.groupby(['key'], as_index=False)['prize_money'].last()

### Merge sectional_time to race_result_full

In [24]:
sectional_time = pd.read_csv('sectional_time.csv')

sectional_time.head()

,finishing_order,horse_no,horse,time,section_time_1,section_time_2,section_time_3,section_time_4,section_time_5,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,date,match
0,1,7,TELEPHATIA(P405),1:49.08,14.91,22.54,23.42,24.31,23.90,NaN,7-3/4,15-1/4,12,5-1/4,SH,NaN,01/01/2015,1
1,2,1,NAMJONG TURBO(N250),1:49.08,13.95,21.86,24.10,24.75,24.42,NaN,1-3/4,5,6,2,SH,NaN,01/01/2015,1
2,3,12,HEAR THE ROAR(M152),1:49.38,14.51,22.30,23.38,24.59,24.60,NaN,5-1/4,11-1/4,7-3/4,2-3/4,2,NaN,01/01/2015,1
3,4,5,CASA JUNIOR(M366),1:49.40,13.79,21.82,23.98,25.15,24.66,NaN,3/4,3-3/4,4,2-1/2,2,NaN,01/01/2015,1
4,5,8,JOYFUL MISSION(S094),1:49.40,14.71,22.02,23.70,24.63,24.34,NaN,6-1/2,10-3/4,9-1/4,4-1/2,2,NaN,01/01/2015,1


In [25]:
sectional_time.loc[:, 'formatted_date'] = pd.to_datetime(sectional_time['date'], format = '%d/%m/%Y')
sectional_time.head()

,finishing_order,horse_no,horse,time,section_time_1,section_time_2,section_time_3,section_time_4,section_time_5,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,date,match,formatted_date
0,1,7,TELEPHATIA(P405),1:49.08,14.91,22.54,23.42,24.31,23.90,NaN,7-3/4,15-1/4,12,5-1/4,SH,NaN,01/01/2015,1,2015-01-01
1,2,1,NAMJONG TURBO(N250),1:49.08,13.95,21.86,24.10,24.75,24.42,NaN,1-3/4,5,6,2,SH,NaN,01/01/2015,1,2015-01-01
2,3,12,HEAR THE ROAR(M152),1:49.38,14.51,22.30,23.38,24.59,24.60,NaN,5-1/4,11-1/4,7-3/4,2-3/4,2,NaN,01/01/2015,1,2015-01-01
3,4,5,CASA JUNIOR(M366),1:49.40,13.79,21.82,23.98,25.15,24.66,NaN,3/4,3-3/4,4,2-1/2,2,NaN,01/01/2015,1,2015-01-01
4,5,8,JOYFUL MISSION(S094),1:49.40,14.71,22.02,23.70,24.63,24.34,NaN,6-1/2,10-3/4,9-1/4,4-1/2,2,NaN,01/01/2015,1,2015-01-01


In [26]:
# get the horse name before parentheses
sectional_time['short_key'] = sectional_time['horse'].str.extract('\(([^)]+)') 
sectional_time['short_key'] = sectional_time['short_key'] + '_' + sectional_time['formatted_date'].astype(str)

sectional_time.head()

,finishing_order,horse_no,horse,time,section_time_1,section_time_2,section_time_3,section_time_4,section_time_5,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,date,match,formatted_date,short_key
0,1,7,TELEPHATIA(P405),1:49.08,14.91,22.54,23.42,24.31,23.90,NaN,7-3/4,15-1/4,12,5-1/4,SH,NaN,01/01/2015,1,2015-01-01,P405_2015-01-01
1,2,1,NAMJONG TURBO(N250),1:49.08,13.95,21.86,24.10,24.75,24.42,NaN,1-3/4,5,6,2,SH,NaN,01/01/2015,1,2015-01-01,N250_2015-01-01
2,3,12,HEAR THE ROAR(M152),1:49.38,14.51,22.30,23.38,24.59,24.60,NaN,5-1/4,11-1/4,7-3/4,2-3/4,2,NaN,01/01/2015,1,2015-01-01,M152_2015-01-01
3,4,5,CASA JUNIOR(M366),1:49.40,13.79,21.82,23.98,25.15,24.66,NaN,3/4,3-3/4,4,2-1/2,2,NaN,01/01/2015,1,2015-01-01,M366_2015-01-01
4,5,8,JOYFUL MISSION(S094),1:49.40,14.71,22.02,23.70,24.63,24.34,NaN,6-1/2,10-3/4,9-1/4,4-1/2,2,NaN,01/01/2015,1,2015-01-01,S094_2015-01-01


In [27]:
# get the horse name before parentheses
race_result_full['short_key'] = race_result_full['Horse'].str.extract('\(([^)]+)') 
race_result_full['short_key'] = race_result_full['short_key'] + '_' + race_result_full['formatted_date'].astype(str)

race_result_full.head()

,Plc,Horse No,Horse,Jockey,Trainer,ActualWt,Declar_HorseWt,Draw,LBW,RunningPosition,FinishTime,Win_Odds,date,match,cdr,prize_money,horseid,formatted_date,key,short_key
0,1,7.0,TELEPHATIA(P405),M L Yeung,A Lee,123,1049,13,,12 13 13 ...,1:49.08,54,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2012_P405,2015-01-01,HK_2012_P405_2015-01-01,P405_2015-01-01
1,2,1.0,NAMJONG TURBO(N250),K C Leung,P O'Sullivan,129,1079,11,SH,6 6 7 ...,1:49.08,5.4,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2011_N250,2015-01-01,HK_2011_N250_2015-01-01,N250_2015-01-01
2,3,12.0,HEAR THE ROAR(M152),M Guyon,S Woods,118,1192,1,2,9 10 9 ...,1:49.38,3,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2010_M152,2015-01-01,HK_2010_M152_2015-01-01,M152_2015-01-01
3,4,5.0,CASA JUNIOR(M366),K Teetan,T K Ng,127,1062,9,2,5 5 5 ...,1:49.40,55,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2010_M366,2015-01-01,HK_2010_M366_2015-01-01,M366_2015-01-01
4,5,8.0,JOYFUL MISSION(S094),O Doleuze,R Gibson,123,1091,2,2,10 9 10 ...,1:49.40,32,2015/01/01,1,Class 5 - 1800M - (40-0),"HK$ 575,000",HK_2013_S094,2015-01-01,HK_2013_S094_2015-01-01,S094_2015-01-01


In [28]:
# drop invalid place in race_result_full since those will not have sectional time or margin behind in sectiontal time
print(race_result_full['Plc'].value_counts())

# Note: You can drop any invalid races in race_result but not in form_record
# because we need to keep the full record of every horses to do feature engineering

# Preserve DH for prize money calculation: need to divide in portion
race_result_full['Plc_PM'] = race_result_full['Plc']

1        9808
2        9781
3        9768
6        9751
4        9736
5        9735
7        9720
8        9656
9        9530
10       9300
11       8914
12       8133
13       4055
14       3397
WV       1949
WV A      385
4 DH      119
WX         94
PU         84
UR         79
3 DH       72
5 DH       64
6 DH       58
WX A       58
2 DH       54
7 DH       48
1 DH       46
8 DH       38
9 DH       36
10 DH      34
FE         28
DNF        26
11 DH      20
TNP        17
DISQ       14
12 DH       4
WXNR        2
Name: Plc, dtype: int64


In [29]:
# Horse that in the same finishing position
race_result_full['Plc'] = race_result_full['Plc'].str.replace(' DH', '')

# drop invalid Place
race_result_full = race_result_full.loc[race_result_full['Plc'].str.contains('^[0-9]+$')]

race_result_full['Plc'].value_counts()

4     9855
1     9854
3     9840
2     9835
6     9809
5     9799
7     9768
8     9694
9     9566
10    9334
11    8934
12    8137
13    4055
14    3397
Name: Plc, dtype: int64

In [30]:
# same for sectional_time
print(sectional_time['finishing_order'].value_counts())

# Horse that in the same finishing position
sectional_time['finishing_order'] = sectional_time['finishing_order'].str.replace(' DH', '')
sectional_time['finishing_order'] = sectional_time['finishing_order'].str.replace(' ', '') #fixing the format

# drop invalid Place
sectional_time = sectional_time.loc[sectional_time['finishing_order'].str.contains('^[0-9]+$')]

sectional_time['finishing_order'].value_counts()

1        9770
2        9748
3        9736
6        9719
4        9704
5        9701
7        9689
8        9626
9        9500
10       9268
11       8887
12       8109
13       4029
14       3380
 WV       616
 WV-A     383
4 DH      121
 PU        84
 UR        78
3 DH       70
5 DH       64
 WX-A      58
6 DH       58
2 DH       54
1 DH       46
7 DH       46
 WX        43
8 DH       38
9 DH       36
10 DH      34
 FE        28
 DNF       26
11 DH      20
 TNP       17
 DISQ      14
12 DH       4
            4
 WXNR       2
Name: finishing_order, dtype: int64


4     9825
1     9816
3     9806
2     9802
6     9777
5     9765
7     9735
8     9664
9     9536
10    9302
11    8907
12    8113
13    4029
14    3380
Name: finishing_order, dtype: int64

In [31]:
# what race_result_full has but sectional_time hasn't
sectional_time_miss = list(set(race_result_full['short_key'].unique()) - set(sectional_time['short_key'].unique()))

# what sectional_time has but race_result_full hasn't
race_result_miss = list(set(sectional_time['short_key'].unique()) - set(race_result_full['short_key'].unique()))

'''after dropping'''
# should only contain races that are too old that have no record of running position
print('should only contain races that are too old that have no record of running position')
print('sectional time has to crawl on:\n', list(race_result_full.loc[race_result_full['short_key'].isin(sectional_time_miss)]['formatted_date'].dt.strftime('%d/%m/%Y').unique()))

# it should be blank
print('\nrace_result has to crawl on:\n', list(sectional_time.loc[sectional_time['short_key'].isin(race_result_miss)]['formatted_date'].dt.strftime('%Y/%m/%d').unique()))

# for checking
# race_result_full.loc[race_result_full['short_key'].isin(sectional_time_miss)]
# sectional_time.loc[sectional_time['short_key'].isin(race_result_miss)]

#race_result_full.loc[race_result_full['short_key'].isin(sectional_time_miss)][['Plc', 'date']]

sectional time has to crawl on:
 ['15/12/2007', '09/02/2008', '09/03/2008']

race_result has to crawl on:
 []


In [32]:
# Fix the weird cases
# https://racing.hkjc.com/racing/information/english/Racing/DisplaySectionalTime.aspx?RaceDate=03/03/2018&RaceNo=5
# https://racing.hkjc.com/racing/information/english/Racing/DisplaySectionalTime.aspx?RaceDate=25/06/2017&RaceNo=3

sectional_time[sectional_time['section_time_2'].isnull()]

,finishing_order,horse_no,horse,time,section_time_1,section_time_2,section_time_3,section_time_4,section_time_5,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,date,match,formatted_date,short_key
25770,5,1,PAKISTAN STAR(V325),2:47.10,15.23,NaN,NaN,NaN,NaN,NaN,6-1/4,TO,NaN,NaN,NaN,NaN,25/06/2017,3,2017-06-25,V325_2017-06-25
32229,13,4,BEAUTY DAY(B076),1:44.53,28.21,NaN,NaN,NaN,NaN,NaN,28,57-1/4,NaN,NaN,NaN,NaN,03/03/2018,5,2018-03-03,B076_2018-03-03


In [33]:
# Switch values

sectional_time.loc[sectional_time['short_key'] == 'V325_2017-06-25',['margin_behind_2','margin_behind_6']] = sectional_time.loc[sectional_time['short_key'] == 'V325_2017-06-25',['margin_behind_6','margin_behind_2']].values

sectional_time.loc[sectional_time['short_key'] == 'B076_2018-03-03',['section_time_1','section_time_4']] = sectional_time.loc[sectional_time['short_key'] == 'B076_2018-03-03',['section_time_4','section_time_1']].values
sectional_time.loc[sectional_time['short_key'] == 'B076_2018-03-03',['margin_behind_1','margin_behind_2', 'margin_behind_5', 'margin_behind_6']] = sectional_time.loc[sectional_time['short_key'] == 'B076_2018-03-03',['margin_behind_5','margin_behind_6', 'margin_behind_1', 'margin_behind_2']].values

In [34]:
display(sectional_time.loc[sectional_time['short_key'] == 'V325_2017-06-25'])
display(sectional_time.loc[sectional_time['short_key'] == 'B076_2018-03-03'])

,finishing_order,horse_no,horse,time,section_time_1,section_time_2,section_time_3,section_time_4,section_time_5,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,date,match,formatted_date,short_key
25770,5,1,PAKISTAN STAR(V325),2:47.10,15.23,NaN,NaN,NaN,NaN,NaN,6-1/4,NaN,NaN,NaN,NaN,TO,25/06/2017,3,2017-06-25,V325_2017-06-25


,finishing_order,horse_no,horse,time,section_time_1,section_time_2,section_time_3,section_time_4,section_time_5,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,date,match,formatted_date,short_key
32229,13,4,BEAUTY DAY(B076),1:44.53,NaN,NaN,NaN,28.21,NaN,NaN,NaN,NaN,NaN,NaN,28,57-1/4,03/03/2018,5,2018-03-03,B076_2018-03-03


### merge sectional time to race result full

In [35]:
race_result_full.shape

(121877, 21)

In [36]:
sectional_time.shape

(121457, 20)

In [37]:
# = race_result_full.loc[race_result_full['short_key'].isin(sectional_time_miss)]
race_result_full.shape[0] - sectional_time.shape[0]

420

In [38]:
# merge
race_result_full_leftjoin_st = race_result_full.merge(sectional_time, on = 'short_key', how = 'left', suffixes = ('', '_dup'))

# check
race_result_full_leftjoin_st.shape

(121877, 40)

In [39]:
# drop duplicate columns

race_result_full_leftjoin_st.drop(list(race_result_full_leftjoin_st.filter(regex = '_dup')), axis = 1, inplace = True)

In [40]:
# get the participating horses here for normalization

race_result_full_leftjoin_st['participating_horse'] = race_result_full_leftjoin_st.groupby(['date', 'match'])['Plc'].transform('size')

### merge participating horses and prize_money to form_record (also sectional time information)

In [41]:
form_record_leftjoin = form_record.merge(race_result_full_leftjoin_st, how = 'left', on = 'key', suffixes = ('', '_dup'))

In [42]:
form_record_leftjoin.columns

Index(['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
       'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
       'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'Gear',
       'VideoReplay', 'horseid', 'formatted_date', 'key', 'Plc', 'Horse No',
       'Horse', 'Jockey_dup', 'Trainer_dup', 'ActualWt', 'Declar_HorseWt',
       'Draw_dup', 'LBW_dup', 'RunningPosition', 'FinishTime_dup', 'Win_Odds',
       'date', 'match', 'cdr', 'prize_money', 'horseid_dup',
       'formatted_date_dup', 'short_key', 'Plc_PM', 'finishing_order',
       'horse_no', 'horse', 'time', 'section_time_1', 'section_time_2',
       'section_time_3', 'section_time_4', 'section_time_5', 'section_time_6',
       'margin_behind_1', 'margin_behind_2', 'margin_behind_3',
       'margin_behind_4', 'margin_behind_5', 'margin_behind_6',
       'participating_horse'],
      dtype='object')

In [43]:
form_record.shape

(83773, 22)

In [44]:
form_record_leftjoin.shape

(83773, 59)

In [45]:
# Merge dropped prize money back to form_record (due to invalid entry in race result)
form_record_leftjoin = form_record_leftjoin.merge(prize_money_table, on = ['key'], how='left', suffixes=('_less_invalid', '_after_2015'))

form_record_leftjoin['prize_money'] = form_record_leftjoin['prize_money_after_2015'].fillna(form_record_leftjoin['prize_money_less_invalid'])

# check any missing prize_money
form_record_leftjoin.loc[(form_record_leftjoin['prize_money'].isna())]

,RaceIndex,Pla,Date,RC/Track/Course,Dist,Ground,RaceClass,Draw,Rating,Trainer,...,section_time_6,margin_behind_1,margin_behind_2,margin_behind_3,margin_behind_4,margin_behind_5,margin_behind_6,participating_horse,prize_money_after_2015,prize_money


In [46]:
# Export for feature engineering

In [47]:
cd

/Users/lauchunlok


In [48]:
form_record_leftjoin.to_csv('HKJC-feature-engineering/form_record_clean.csv', index=False)